In [1]:
%%capture
!pip install adabelief-tf --no-cache-dir 
from adabelief_tf import AdaBeliefOptimizer

In [2]:
import pandas as pd
import numpy as np
import dill as pickle   
from matplotlib import pyplot as plt
import random
import datetime
import math
import gc
import os
import warnings
import seaborn as sns
import itertools
import multiprocessing
import joblib
warnings.simplefilter(action='ignore', category=FutureWarning)
import pickle
from pickle import load,dump
from tensorflow.keras.layers import LSTM

from matplotlib.ticker import MaxNLocator
from colorama import Fore, Back, Style
from tqdm import tqdm
import h5py

from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Bidirectional

from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.preprocessing import StandardScaler, QuantileTransformer, OneHotEncoder, PowerTransformer
from sklearn.metrics import roc_curve, roc_auc_score, average_precision_score
from sklearn.impute import SimpleImputer
from sklearn.utils import class_weight 
from sklearn.utils.class_weight import compute_class_weight
from keras.layers import CuDNNLSTM
import tensorflow_addons as tfa
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
tf.config.threading.set_inter_op_parallelism_threads(4)
from tensorflow import keras
from tensorflow.keras.models import Model, load_model,model_from_json
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Add, Concatenate, Dropout, BatchNormalization, Conv1D, Reshape, Flatten, AveragePooling1D, MaxPool1D
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import binary_crossentropy
import tensorflow.keras.backend as K

In [3]:
ss = load(open('../input/standardscaler/standardscaler.pkl','rb'))

In [4]:
def amex_metric(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)
    
class MyCustomMetricCallback(tf.keras.callbacks.Callback):

    def __init__(self, save_path, train=None, validation=None):
        super(MyCustomMetricCallback, self).__init__()
        self.train = train
        self.validation = validation
        self.best_score = 0
        self.save_path =save_path

    def on_epoch_end(self, epoch, logs={}):
        if self.train:
            pass

        if self.validation:
            X_valid, y_valid = self.validation[0], self.validation[1]
            y_pred = self.model.predict(X_valid).reshape( (len(X_valid), )) 
            val_score = amex_metric(y_valid, y_pred)
            logs['my_metric_val'] = val_score
            print(val_score)
            if val_score>self.best_score:
                self.y_valid = y_valid
                self.best_pred = y_pred
                self.best_epoch = epoch
                self.best_score = val_score
                self.model.save(f"{self.save_path}.h5")
                print('best_val_score: ', self.best_score)
            elif epoch-self.best_epoch > 10:
                self.model.stop_training = True    
            del X_valid, y_valid, y_pred, val_score
            gc.collect()
            

ALPHA= 5
GAMMA = 2
def FocalLoss(targets, inputs, alpha=ALPHA, gamma=GAMMA):    
    BCE = K.binary_crossentropy(targets, inputs)
    BCE_EXP = K.exp(-BCE)
    focal_loss = K.mean(alpha * K.pow((1-BCE_EXP), gamma) * BCE)
    
    return focal_loss

In [5]:
def my_model(n_inputs):
    """Sequential neural network with a skip connection.
    
    Returns a compiled instance of tensorflow.keras.models.Model.
    """
    activation = 'elu'
    inputs = Input(shape=(n_inputs, ))
    x = Reshape((n_inputs, 1))(inputs)
    x = keras.layers.Conv1D(32,1,strides=1, activation=activation)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv1D(24,1, activation=activation)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv1D(16,1, activation=activation)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv1D(4,1, activation=activation)(x)
   # x = LSTM(64,return_sequences=True)(x)
    x = Flatten()(x)
    x = Dropout(0.3)(x)
    x = Dense(16, activation = activation)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)
    x = Dense(8, activation = activation)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    gc.collect()
    return Model(inputs, outputs)
model = my_model(770)
model.summary()
del model

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 770)]             0         
_________________________________________________________________
reshape (Reshape)            (None, 770, 1)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 770, 32)           64        
_________________________________________________________________
batch_normalization (BatchNo (None, 770, 32)           128       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 770, 24)           792       
_________________________________________________________________
batch_normalization_1 (Batch (None, 770, 24)           96        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 770, 16)           400   

In [6]:
VERBOSE = 1
CYCLES = 1
EPOCHS = 30
BATCH_SIZE = 2048
FOLDS = 5
SEED = 42
CURRENT_FOLD = 0
val_pred = []
target = []

def fit_model(seed, fold):
    best_score = 0
    train = pd.read_feather('../input/amexfeatureengineering/770_FE_train.feather')
    FEATURES = [col for col in train.columns if col not in ['customer_ID','target']]
    target = train.target.astype('float32')
    train = train.loc[:,FEATURES]
    
    train = ss.transform(train)
    
    train = np.array(train)
    gc.collect()
    
    
    idx_tr, idx_va = list(StratifiedKFold(n_splits=FOLDS, shuffle= True, random_state= seed).split(train,target))[fold]
    X_va = train[idx_va]
    X_tr = train[idx_tr]
    y_tr, y_va = target[idx_tr], target[idx_va]
    del train, target, idx_tr
    gc.collect()
    
    custom = MyCustomMetricCallback(save_path = f'model_fold{fold}_seed{seed}', validation=(X_va, y_va))
    
    lr = ReduceLROnPlateau(monitor='my_metric_val',
                           factor=0.2, 
                           patience=7, 
                           mode = 'max', 
                           verbose=VERBOSE)
    callbacks = [lr, 
                 tf.keras.callbacks.TerminateOnNaN(), 
                 custom
                ]
    model = my_model(X_tr.shape[1])
    model.compile(optimizer=AdaBeliefOptimizer(learning_rate=0.02,
                                               weight_decay = 1e-5,
                                               epsilon = 1e-7,
        ),
        loss=FocalLoss,
        )
    
    oof = np.zeros((458913,))
    best_loss = np.inf
    
    gc.collect()
    model.fit(X_tr, y_tr, 
            validation_data=(X_va, y_va),
            epochs=EPOCHS,
            verbose=VERBOSE,
            batch_size=BATCH_SIZE,
            shuffle=True,
            callbacks=callbacks)
    
    oof[idx_va] =  custom.best_pred
    
    print(f'Best Amex Score = {custom.best_score}')
    return oof

In [7]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((458913,))
    
    for fold in range(FOLDS):
        print(f'Fold {fold}_SEED {seed}')
        oof_ = fit_model(seed, fold)
        oof += oof_
    return oof

In [8]:
SEED = [41,42]  #<-- Update
oof = np.zeros((458913,))

for seed in SEED:
    
    oof_ = run_k_fold(FOLDS, seed)
    oof += oof_ / len(SEED)

Fold 0_SEED 41
Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



2022-08-12 04:30:37.341101: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1130760400 exceeds 10% of free system memory.
2022-08-12 04:30:38.728206: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1130760400 exceeds 10% of free system memory.


Epoch 1/30
180/180 [==============================] - 29s 92ms/step - loss: 0.4023 - val_loss: 0.3355
0.7744368536692271
best_val_score:  0.7744368536692271
Epoch 2/30
180/180 [==============================] - 15s 86ms/step - loss: 0.3017 - val_loss: 0.3223
0.7724656782483363
Epoch 3/30
180/180 [==============================] - 16s 87ms/step - loss: 0.2976 - val_loss: 0.3131
0.777276986114882
best_val_score:  0.777276986114882
Epoch 4/30
180/180 [==============================] - 15s 85ms/step - loss: 0.2954 - val_loss: 0.3111
0.7865420080968879
best_val_score:  0.7865420080968879
Epoch 5/30
180/180 [==============================] - 15s 86ms/step - loss: 0.2935 - val_loss: 0.2928
0.7884580350115343
best_val_score:  0.7884580350115343
Epoch 6/30
180/180 [==============================] - 16s 88ms/step - loss: 0.2931 - val_loss: 0.2937
0.788996288983147
best_val_score:  0.788996288983147
Epoch 7/30
180/180 [==============================] - 15s 85ms/step - loss: 0.2913 - val_loss: 0.2

2022-08-12 04:43:22.848408: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1130760400 exceeds 10% of free system memory.
2022-08-12 04:43:24.218826: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1130760400 exceeds 10% of free system memory.


Epoch 1/30
180/180 [==============================] - 21s 93ms/step - loss: 0.3369 - val_loss: 0.4436
0.7600466085195234
best_val_score:  0.7600466085195234
Epoch 2/30
180/180 [==============================] - 16s 88ms/step - loss: 0.2988 - val_loss: 0.3649
0.7769488854295965
best_val_score:  0.7769488854295965
Epoch 3/30
180/180 [==============================] - 15s 85ms/step - loss: 0.2956 - val_loss: 0.3044
0.7832747207679059
best_val_score:  0.7832747207679059
Epoch 4/30
180/180 [==============================] - 16s 88ms/step - loss: 0.2935 - val_loss: 0.2980
0.7845305524537263
best_val_score:  0.7845305524537263
Epoch 5/30
180/180 [==============================] - 15s 85ms/step - loss: 0.2923 - val_loss: 0.2972
0.7841916247776761
Epoch 6/30
180/180 [==============================] - 15s 85ms/step - loss: 0.2912 - val_loss: 0.3012
0.7834328797228798
Epoch 7/30
180/180 [==============================] - 15s 86ms/step - loss: 0.2903 - val_loss: 0.2942
0.7850085875030598
best_val_

2022-08-12 04:55:00.028655: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1130760400 exceeds 10% of free system memory.


Epoch 1/30
180/180 [==============================] - 20s 90ms/step - loss: 0.3751 - val_loss: 0.3413
0.7686489635882168
best_val_score:  0.7686489635882168
Epoch 2/30
180/180 [==============================] - 16s 87ms/step - loss: 0.3022 - val_loss: 0.5107
0.7060599611636136
Epoch 3/30
180/180 [==============================] - 16s 87ms/step - loss: 0.2975 - val_loss: 0.3021
0.7882512796309551
best_val_score:  0.7882512796309551
Epoch 4/30
180/180 [==============================] - 16s 87ms/step - loss: 0.2953 - val_loss: 0.3157
0.7882113457316522
Epoch 5/30
180/180 [==============================] - 16s 87ms/step - loss: 0.2940 - val_loss: 0.2930
0.7920114426206408
best_val_score:  0.7920114426206408
Epoch 6/30
180/180 [==============================] - 15s 85ms/step - loss: 0.2929 - val_loss: 0.2884
0.7936190973320469
best_val_score:  0.7936190973320469
Epoch 7/30
180/180 [==============================] - 16s 87ms/step - loss: 0.2918 - val_loss: 0.3188
0.7884326963835735
Epoch 8/3

In [9]:
train = pd.read_feather('../input/amexfeatureengineering/770_FE_train.feather')

In [10]:
target = train.target

In [11]:
oof_Amex = amex_metric(target,oof)

In [12]:
oof_Amex

0.7938481749446653

In [13]:
oof

array([0.61192226, 0.04923945, 0.06925567, ..., 0.68758154, 0.16656805,
       0.66501907])

In [14]:
oof = pd.DataFrame({'customer_ID':train.customer_ID,'target':train.target,'oof_pred':oof})
oof.to_csv('oof_CNN1D.csv',index=False)